# Querying with Athena

In [1]:
import boto3
session = boto3.Session()
athena = session.client('athena')

In [2]:
response = athena.start_query_execution(
    QueryString='select * from jessica.transactions',
    QueryExecutionContext={
        'Database': 'jessica',
    },
    ResultConfiguration={
        'OutputLocation': 's3://bigdata-dashboards-develop/query_results/',
    }
)
response

{'QueryExecutionId': 'a13aee63-ea35-42d5-b7d6-e97cf46f9d9e',
 'ResponseMetadata': {'RequestId': '1a393604-eb1a-46d7-8a61-14705297aed5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 01 May 2021 00:46:44 GMT',
   'x-amzn-requestid': '1a393604-eb1a-46d7-8a61-14705297aed5',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Results using Athena

In [3]:
results = athena.get_query_results(
    QueryExecutionId=response['QueryExecutionId']
)
results

{'UpdateCount': 0,
 'ResultSet': {'Rows': [{'Data': [{'VarCharValue': 'customer_is'},
     {'VarCharValue': 'seller_id'},
     {'VarCharValue': 'prov_cod'},
     {'VarCharValue': 'datestamp'},
     {'VarCharValue': 'quantity'},
     {'VarCharValue': 'total_price'},
     {'VarCharValue': 'gross_profit'},
     {'VarCharValue': 'total_cost'}]},
   {'Data': [{'VarCharValue': '3'},
     {'VarCharValue': '3'},
     {'VarCharValue': '5'},
     {'VarCharValue': '2020-03-04'},
     {'VarCharValue': '4.5'},
     {'VarCharValue': '63.0'},
     {'VarCharValue': '34.4'},
     {'VarCharValue': '0.23'}]},
   {'Data': [{'VarCharValue': '3'},
     {'VarCharValue': '3'},
     {'VarCharValue': '5'},
     {'VarCharValue': '2019-03-04'},
     {'VarCharValue': '4.5'},
     {'VarCharValue': '63.0'},
     {'VarCharValue': '34.4'},
     {'VarCharValue': '0.23'}]},
   {'Data': [{'VarCharValue': '1'},
     {'VarCharValue': '1'},
     {'VarCharValue': '1'},
     {'VarCharValue': '2019-02-01'},
     {},
     {},
 

## Results using S3

In [4]:
import pandas as pd
s3 = session.client('s3')

In [5]:
obj = s3.get_object(Bucket='bigdata-dashboards-develop', 
                    Key=f"query_results/{response['QueryExecutionId']}.csv")

In [6]:
df = pd.read_csv(obj['Body'])

In [7]:
df

,customer_is,seller_id,prov_cod,datestamp,quantity,total_price,gross_profit,total_cost
0,3,3,5,2020-03-04,4.5,63.0,34.4,0.23
1,3,3,5,2019-03-04,4.5,63.0,34.4,0.23
2,1,1,1,2019-02-01,NaN,NaN,NaN,NaN
3,1,1,1,2019-02-01,4.3,4.3,4.3,4.30
4,3,3,5,2020-03-04,4.5,63.0,34.4,0.23


## Inserts 

In [8]:
response = athena.start_query_execution(
    QueryString='''
        INSERT INTO jessica.transactions 
        VALUES (3, 3, 5, '2020-03-04', 4.5, 63, 34.4, .23)
    ''',
    QueryExecutionContext={
        'Database': 'jessica',
    },
    ResultConfiguration={
        'OutputLocation': 's3://bigdata-dashboards-develop/query_results/',
    }
)
response

{'QueryExecutionId': '1d00738b-d38c-403e-8547-2f6bdfce9857',
 'ResponseMetadata': {'RequestId': 'a7208327-ed9a-4ec8-91a5-bb47b2c1b42b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 01 May 2021 00:46:59 GMT',
   'x-amzn-requestid': 'a7208327-ed9a-4ec8-91a5-bb47b2c1b42b',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [9]:
response = athena.get_query_execution(
    QueryExecutionId=response['QueryExecutionId']
)
response

{'QueryExecution': {'QueryExecutionId': '1d00738b-d38c-403e-8547-2f6bdfce9857',
  'Query': "INSERT INTO jessica.transactions \n        VALUES (3, 3, 5, '2020-03-04', 4.5, 63, 34.4, .23)",
  'StatementType': 'DML',
  'ResultConfiguration': {'OutputLocation': 's3://bigdata-dashboards-develop/query_results/1d00738b-d38c-403e-8547-2f6bdfce9857'},
  'QueryExecutionContext': {'Database': 'jessica'},
  'Status': {'State': 'SUCCEEDED',
   'SubmissionDateTime': datetime.datetime(2021, 5, 1, 0, 46, 59, 955000, tzinfo=tzlocal()),
   'CompletionDateTime': datetime.datetime(2021, 5, 1, 0, 47, 1, 916000, tzinfo=tzlocal())},
  'Statistics': {'EngineExecutionTimeInMillis': 1351,
   'DataScannedInBytes': 0,
   'DataManifestLocation': 's3://bigdata-dashboards-develop/query_results/1d00738b-d38c-403e-8547-2f6bdfce9857-manifest.csv',
   'TotalExecutionTimeInMillis': 1961,
   'QueryQueueTimeInMillis': 423,
   'QueryPlanningTimeInMillis': 242,
   'ServiceProcessingTimeInMillis': 187},
  'WorkGroup': 'primar

In [10]:
response = athena.start_query_execution(
    QueryString='select * from jessica.transactions',
    QueryExecutionContext={
        'Database': 'jessica',
    },
    ResultConfiguration={
        'OutputLocation': 's3://bigdata-dashboards-develop/query_results/',
    }
)

# Wait until it is done
while athena.get_query_execution(
    QueryExecutionId=response['QueryExecutionId']
)['QueryExecution']['Status']['State'] != 'SUCCEEDED':
    pass

# Get DF
obj = s3.get_object(Bucket='bigdata-dashboards-develop', 
                    Key=f"query_results/{response['QueryExecutionId']}.csv")
df = pd.read_csv(obj['Body'])
df

,customer_is,seller_id,prov_cod,datestamp,quantity,total_price,gross_profit,total_cost
0,1,1,1,2019-02-01,NaN,NaN,NaN,NaN
1,3,3,5,2020-03-04,4.5,63.0,34.4,0.23
2,1,1,1,2019-02-01,4.3,4.3,4.3,4.30
3,3,3,5,2020-03-04,4.5,63.0,34.4,0.23
4,3,3,5,2019-03-04,4.5,63.0,34.4,0.23
5,3,3,5,2020-03-04,4.5,63.0,34.4,0.23
